# ASSIGNMENT: Cartoon Face Mask
# Michael McAllister
## This assignment tests your ability to apply face recognition, image masks, image thresholding, video capturing and saving video feed into a video file concepts.

## TASK: Cartoonify faces in video feed from live webcam

### Steps
- 1. **Capture video** feed from webcam
- 2. **Recognize faces** in the video
- 3. **Replace/Mask the face** region with your favorite cartoon character
- 4. **Save the video** feed into a video file
- 5. Submit the following files
    - Notebook (.ipynb)
    - Video (.avi/.mp4)

### Helper code to recognize faces

In [11]:
import cv2
import os
cascPathface = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
cascPatheyes = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_eye_tree_eyeglasses.xml"

faceCascade = cv2.CascadeClassifier(cascPathface)
eyeCascade = cv2.CascadeClassifier(cascPatheyes)

cartoon = cv2.imread("cartoon_face.jpg")

video_capture = cv2.VideoCapture(0)

# get camera width
width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
# get camera height
height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
# define the name of the ouput video
filename = "output.avi"
# FPS
fps = 24
# video encoder used
fourcc = cv2.VideoWriter_fourcc(*"XVID")
# define the output
out=cv2.VideoWriter(filename, fourcc, fps, (width, height)) 


while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h),(0,255,0), 2)
        faceROI = frame[y:y+h,x:x+w]
        eyes = eyeCascade.detectMultiScale(faceROI)
        for (x2, y2, w2, h2) in eyes:
            eye_center = (x + x2 + w2 // 2, y + y2 + h2 // 2)
            radius = int(round((w2 + h2) * 0.25))
            frame = cv2.circle(frame, eye_center, radius, (255, 0, 0), 4)
            
    # Reshape cartoon face to the shape of faceROI
    cartoon_resized = cv2.resize(cartoon, (int(faceROI.shape[1]),int(faceROI.shape[0])), interpolation = cv2.INTER_AREA)
    
    # Replace face ROI with cartoon
    background = frame.copy()
    foreground = cartoon_resized.copy()
    x_offset= x
    y_offset= y
    
    # Impose shifted/offsetted foreground onto background 
    background[y_offset:y_offset+foreground.shape[0], x_offset:x_offset+foreground.shape[1]] = foreground 
    
    # If ret is equal to true, write the background
    if ret == True:
        out.write(background)             

    # Display the resulting frame
    cv2.imshow('Face Video', frame)
    cv2.imshow("Face ROI", faceROI)

    #q to stop recording
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

In [10]:
video_capture.release()